In [26]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import sklearn
from sklearn.naive_bayes import GaussianNB
import collections
from nltk.stem import PorterStemmer
import re
import contractions

In [27]:
chitchat_df = pd.read_pickle("./chitchat.pkl")
chitchat_df.drop(['prompt'], axis=1, inplace=True)
cc_new = pd.DataFrame().assign(X=chitchat_df['query'])
cc_new["Y"] = 0
cc_new = cc_new[["Y" ,"X"]]

In [28]:
chitchat_df

,query,response
0,Hello,whats up MD
1,How are you doing today?,im doing good
2,whats up MD,"Im alright, I just took a nap. But it was one ..."
3,"Im alright, I just took a nap. But it was one ...",oh wow haha
4,oh wow haha,Yeah
...,...,...
166163,12 miles is solid.,"Thanks, it wasn't too bad."
166164,"Oh man that's too bad, I'll take lots of pictu...","Hey as long as you have water and time, the hi..."
166165,"Hey as long as you have water and time, the hi...","That is so true, I hear you can pay like three..."
166166,"That is so true, I hear you can pay like three...",Only problem is I don't have the money for tha...


In [29]:
reddit_df = pd.read_pickle("./final.pkl")
reddit_df = reddit_df.loc[reddit_df['is_submission'] == False][["parent_body","body"]]
reddit_df.rename(columns={"parent_body": "query", "body": "response"}, inplace=True)
merged_qr_df = pd.concat([chitchat_df, reddit_df])

In [30]:
reddit_new = pd.DataFrame().assign(X=reddit_df['query'])
reddit_new["Y"] = 1
reddit_new = reddit_new[["Y", "X"]]
# reddit_df.loc[reddit_df['is_submission'] == False]["body"]
res_df = pd.concat([reddit_new, cc_new])
res_df.drop_duplicates(inplace=True)

In [31]:
res_df

,Y,X
0,1,Im banned already when I called a racist Karen...
2,1,I have fallen for this trap several times and ...
6,1,Reddit is nearly unusable because of the troll...
8,1,this has pretty much been my experience in all...
12,1,I got banned from permanently from r/politics ...
...,...,...
166163,0,12 miles is solid.
166164,0,"Oh man that's too bad, I'll take lots of pictu..."
166165,0,"Hey as long as you have water and time, the hi..."
166166,0,"That is so true, I hear you can pay like three..."


In [32]:
def cleaner(text):    
    text = contractions.fix(text)
    text = text.replace("&gt;", "")
    text = text.replace("&lt;", "")
    text = text.replace("\n", " ")

    regex_url = r"(http[^\s]+)"
    wo_url = re.sub(regex_url,"",text)
    wo_url = re.sub(r"#","",wo_url)
    # wo_url = re.sub(r"(@[^\s]+)","",wo_url)
    wo_url = re.sub(r"(@[^\s]+)","@USER_",wo_url)

    return wo_url



res_df['X'] = res_df['X'].apply(cleaner)

In [33]:
# res_df.to_csv(r'A:\MS2021\SUNY BUFFALO\Third sem\Info Retrieval\Project 4\ir_project_4\backend\classifier\data.csv', header=None, index=None, sep=' ', mode='a')

In [34]:
# from transformers import BertTokenizer
# # using the low level BERT for our task.
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [35]:
merged_qr_df

,query,response
0,Hello,whats up MD
1,How are you doing today?,im doing good
2,whats up MD,"Im alright, I just took a nap. But it was one ..."
3,"Im alright, I just took a nap. But it was one ...",oh wow haha
4,oh wow haha,Yeah
...,...,...
72,That is all.,I came here to see what all the fuss was about...
73,That is all.,That is all. That is all.
74,That is all.,Ditto. I believe that is all.
75,Me too. That is all.,Same. That is all


Data Preprocessing done, now starting with classification
------------------------------------------------------


In [36]:
#Import feature engineering modules and test_train_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

#Import classification algorithm
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
# from xgboost import XGBClassifier

#Import modules to calculate accuracy and confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report


NAIVE BAYES CLASSIFICATION
-----------------------------


In [37]:
tv = TfidfVectorizer(ngram_range=(1,3),max_features=15000) 
X = tv.fit_transform(res_df['X'])
y = res_df["Y"].values
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.2, shuffle=True)
nb = MultinomialNB(alpha=6.5, fit_prior=False)
nb.fit(Xtrain,ytrain)
nb


MultinomialNB(alpha=6.5, fit_prior=False)

ACCURACY FOR MODEL n max_features
----------------------------------
ACCURACY 20 - 0.6478967862235384 
ACCURACY 40 - 0.7042064275529231
ACCURACY 60 - 0.7171492815619173
ACCURACY 80 - 0.7226061204343535
ACCURACY 100 - 0.7327108698036635
ACCURACY 120 - 0.7369885927388395
ACCURACY 140 - 0.7538252714708786
ACCURACY 160 - 0.7513710650433256
ACCURACY 180 - 0.7578973346495558
ACCURACY 200 - 0.7583497861138533
ACCURACY 220 - 0.7634364374245914
ACCURACY 240 - 0.7639574421410552
ACCURACY 260 - 0.774528353625096
ACCURACY 280 - 0.773404080289569
ACCURACY 300 - 0.777201930459581
ACCURACY 350 - 0.781205440386092
ACCURACY 400 - 0.7873341011297575
ACCURACY 450 - 0.7911867939015027
ACCURACY 500 - 0.7937918174838214

1000-1500 step50 
[0.8115196884940221
0.813439179554678
0.8165926291543271
0.817552374684655
0.8168668421629922
0.8146182954919381
0.8197597894044093
0.8182653285071844
0.8207606668860371
0.8221043106284962]


5000 - 5250
[0.6777722935176045
0.8398458922891302
0.839571679280465
0.8414363277393879
0.8435340572556762]

ACCURACY 15000 - 0.8505539102775036
---------------------------------


In [38]:
pred = nb.predict(Xtest)
print(accuracy_score(ytest,pred))


0.8301532864694848


In [39]:
# changes
s = [{"X":"How?"},{"X":"What?"},{"X":"Why?"},{"X":"Don't know"},{"X":"Iphone is slow"},{"X":"I hate joe biden"},{"X":"What is Covid 19"},{"X":"What is"},{"X":"Covid 19"},{"X":"Stock market"},{"X":"Sushi sucks?"}]
df = pd.DataFrame(s)



In [40]:
# tv = TfidfVectorizer(ngram_range=(1,3),max_features=15000) 
XDemo = tv.transform(df['X'])

In [41]:
my_preds = nb.predict(XDemo)
my_preds

array([0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0], dtype=int64)

Before
array([0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0], dtype=int64

After
array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0], dtype=int64)

In [42]:
import pickle

In [43]:
# file name, I'm using *.pickle as a file extension
filename = "model.pickle"

# save model
pickle.dump(nb, open(filename, "wb"))
pickle.dump(tv, open("tv.pickle", "wb"))

# load model
# loaded_model = pickle.load(open(filename, "rb"))

Trying decision tree now
-------------------------

In [ ]:
# dt = DecisionTreeClassifier()
# dt.fit(Xtrain,ytrain)
# pred = dt.predict(Xtest)



In [ ]:
# print(accuracy_score(ytest,pred))
# print(confusion_matrix(ytest,pred))
# print(classification_report(ytest,pred))

Random forest
------------
taking a lot of time


In [ ]:
# rf = RandomForestClassifier()
# rf.fit(Xtrain,ytrain)
# pred = rf.predict(Xtest)


In [ ]:

# print(accuracy_score(ytest,pred))
# print(confusion_matrix(ytest,pred))
# print(classification_report(ytest,pred))